In [ ]:
# !pip install langchain-community
# !pip install langgraph
# !pip install langchain_huggingface

In [ ]:
from langgraph.graph import StateGraph, END

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
#from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
from langchain.tools import TavilySearchResults
import os
os.environ["TAVILY_API_KEY"] = "****"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline,set_seed
import torch

model_name = "Qwen/Qwen3-4B"  # Replace with your local path if needed

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)

In [ ]:
generator = pipeline('text-generation',model=model,tokenizer=tokenizer,
                     return_full_text=False,max_new_tokens=750,temperature=0.6, top_p=0.95, top_k=20, min_p=0)
# def qwen_generate(prompt,generator=generator):
#     with torch.no_grad():
#       set_seed(42)
#       res = generator(prompt)[0]['generated_text']
#     return res

In [ ]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace,HuggingFacePipeline

In [ ]:
llm=HuggingFacePipeline(pipeline=generator)
chat = ChatHuggingFace(llm=llm, model_id="Qwen/Qwen3-4B", verbose=True)

In [ ]:
search_prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
# messages = [ {"role": "system", "content": search_prompt}]
# prompt = tokenizer.apply_chat_template(
#               messages,
#               tokenize=False,
#               add_generation_prompt=True,
#               enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
#               )
#tool = [search_tool]
abot = Agent(chat, [tool], system=search_prompt)

In [ ]:
messages = [HumanMessage(content="who won icc champions trophy in 2025?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result

In [ ]:
messages = [HumanMessage(content="who is current president of USA?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result